In [1]:
%load_ext autoreload
%autoreload 2

# Test penta-R

In [3]:
import molsysmt as msm

In [ ]:
import openmm as omm
import openmm.app as app
from openmm import unit
from mdtraj.reporters import HDF5Reporter
from sys import stdout

In [ ]:
if False:
    molecular_system = msm.build.build_peptide('RRRRRRR')
    molecular_system = msm.build.add_missing_terminal_cappings(molecular_system, N_terminal='ACE', C_terminal='NME')
    msm.structure.center(molecular_system, selection='molecule_type=="peptide"')
    molecular_system = msm.build.solvate(molecular_system, box_shape='truncated octahedral', clearance='14.0 angstroms')
    msm.convert(molecular_system, to_form='hepta-L-Arg_solv.pdb')

In [ ]:
if False:
    pdb = app.PDBFile('hepta-L-Arg_solv.pdb')
    forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')
    system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)
    integrator = omm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 0.002*unit.picoseconds)
    platform = omm.Platform.getPlatformByName('CUDA')
    simulation = app.Simulation(pdb.topology, system, integrator, platform)
    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy()
    h5_reporter = HDF5Reporter('traj.h5', 2500)
    stdout_reporter = app.StateDataReporter(stdout, 5000, step=True,
        potentialEnergy=True, temperature=True)
    simulation.reporters.append(h5_reporter)
    simulation.reporters.append(stdout_reporter)
    simulation.step(50000)
    h5_reporter.close()

In [4]:
msm.get('traj.h5', n_entities=True)

In [5]:
msm.info('traj.h5')

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_peptides,n_structures
file:h5,4720,1527,1519,3,1519,None,1511,7,1,20
